# Implementation

In [16]:
import pandas as pd
import numpy as np
import tpqoa
import fxcmpy
from datetime import datetime, timedelta
import time

## Loading Parameters

In [17]:
import keras
model = keras.models.load_model("DNN_model")

In [18]:
model

In [19]:
import pickle
params = pickle.load(open("params.pkl", "rb"))
mu = params["mu"]
std = params["std"]

In [20]:
mu

EUR_USD       1.156186
returns       0.000002
dir           0.493137
sma           0.000137
boll          0.014627
min          -0.002536
max           0.002411
mom           0.000002
vol           0.000512
dir_lag_1     0.493137
dir_lag_2     0.493137
dir_lag_3     0.493109
dir_lag_4     0.493082
dir_lag_5     0.493082
sma_lag_1     0.000137
sma_lag_2     0.000138
sma_lag_3     0.000138
sma_lag_4     0.000138
sma_lag_5     0.000138
boll_lag_1    0.014692
boll_lag_2    0.014758
boll_lag_3    0.014822
boll_lag_4    0.014901
boll_lag_5    0.015003
min_lag_1    -0.002536
min_lag_2    -0.002537
min_lag_3    -0.002537
min_lag_4    -0.002537
min_lag_5    -0.002537
max_lag_1     0.002411
max_lag_2     0.002410
max_lag_3     0.002410
max_lag_4     0.002410
max_lag_5     0.002410
mom_lag_1     0.000002
mom_lag_2     0.000002
mom_lag_3     0.000002
mom_lag_4     0.000002
mom_lag_5     0.000002
vol_lag_1     0.000512
vol_lag_2     0.000512
vol_lag_3     0.000512
vol_lag_4     0.000512
vol_lag_5  

In [21]:
std

EUR_USD       0.051209
returns       0.000542
dir           0.499960
sma           0.002872
boll          1.441474
min           0.002368
max           0.002249
mom           0.000308
vol           0.000178
dir_lag_1     0.499960
dir_lag_2     0.499960
dir_lag_3     0.499959
dir_lag_4     0.499959
dir_lag_5     0.499959
sma_lag_1     0.002872
sma_lag_2     0.002872
sma_lag_3     0.002872
sma_lag_4     0.002872
sma_lag_5     0.002872
boll_lag_1    1.441498
boll_lag_2    1.441521
boll_lag_3    1.441539
boll_lag_4    1.441568
boll_lag_5    1.441631
min_lag_1     0.002369
min_lag_2     0.002369
min_lag_3     0.002369
min_lag_4     0.002369
min_lag_5     0.002370
max_lag_1     0.002249
max_lag_2     0.002249
max_lag_3     0.002249
max_lag_4     0.002248
max_lag_5     0.002248
mom_lag_1     0.000308
mom_lag_2     0.000308
mom_lag_3     0.000308
mom_lag_4     0.000308
mom_lag_5     0.000308
vol_lag_1     0.000178
vol_lag_2     0.000178
vol_lag_3     0.000178
vol_lag_4     0.000178
vol_lag_5  

## Implemitation to OANDA

In [22]:
class DNNTrader(tpqoa.tpqoa):
    def __init__(self, conf_file, instrument, bar_length, window, lags, model, mu, std, units):
        super().__init__(conf_file)
        self.instrument = instrument
        self.bar_length = pd.to_timedelta(bar_length)
        self.tick_data = pd.DataFrame()
        self.raw_data = None
        self.data = None
        self.last_bar = None
        self.units = units
        self.position = 0
        self.profits = []
        
        # Strategy specs ********************************
        self.window = window
        self.lags = lags
        self.model = model
        self.mu = mu
        self.std = std
        
    def get_most_recent(self, day=5):
        while True:
            time.sleep(2)
            now = datetime.utcnow()
            now = now - timedelta(microseconds=now.microsecond)
            past = now - timedelta(days=day)
            df = self.get_history(
                instrument=self.instrument, start=past, end=now,
                granularity="M5", price="M", localize=False
            ).dropna().to_frame()
            df.rename(columns={"c": self.instrument}, inplace=True)
            df = df.resample(self.bar_length, label="right").last().dropna().iloc[:-1]
            self.raw_data = df.copy()
            self.last_bar = self.raw_data.index[-1]
            if pd.to_datetime(datetime.utcnow()).tz_localize("UTC") - self.last_bar < self.bar_length:
                self.start_time = pd.to_datetime(datetime.utcnow()).tz_localize("UTC")
                break
                
    def on_success(self, time, bid, ask):
        print(self.ticks, end=" ")
        
        recent_tick = pd.to_datetime(time)
        df = pd.DataFrame({self.instrument: (ask + bid) / 2}, index=[recent_tick])
        self.tick_data = self.tick_data.append(df)
        
        if recent_tick - self.last_bar > self.bar_length:
            self.resample_and_join()
            self.define_strategy()
            self.execute_trades()
            
    def resample_and_join(self):
        self.raw_data = self.raw_data.append(
            self.tick_data.resample(self.bar_length, label="right").last().ffill().iloc[:-1]
        )
        self.tick_data = self.tick_data.iloc[-1:]
        self.last_bar = self.raw_data.index[-1]
        
    def define_strategy(self):
        df = self.raw_data.copy()
        
        df = df.append(self.tick_data)  # Append latest tick
        df['returns'] = np.log(df[self.instrument] / df[self.instrument].shift())
        df["dir"] = np.where(df["returns"] > 0, 1, 0)
        df["sma"] = df[self.instrument].rolling(self.window).mean() - df[self.instrument].rolling(150).mean()
        df["boll"] = (df[self.instrument] - df[self.instrument].rolling(self.window).mean()) / df[self.instrument].rolling(self.window).std()
        df["min"] = df[self.instrument].rolling(self.window).min() / df[self.instrument] - 1
        df["max"] = df[self.instrument].rolling(self.window).max() / df[self.instrument] - 1
        df["mom"] = df["returns"].rolling(3).mean()
        df["vol"] = df["returns"].rolling(self.window).std()
        df.dropna(inplace=True)
        
        self.cols = []
        features = ["dir", "sma", "boll", "min", "max", "mom", "vol"]
        
        for f in features:
            for lag in range(1, self.lags + 1):
                col = f"{f}_lag_{lag}"
                df[col] = df[f].shift(lag)
                self.cols.append(col)
        df.dropna(inplace=True)
        
        df_s = (df - self.mu) / self.std    
        df["proba"] = self.model.predict(df_s[self.cols])
        
        df = df.loc[self.start_time].copy()
        df["position"] = np.where(df["proba"] < 0.47, -1, np.nan)
        df["position"] = np.where(df["proba"] > 0.53, 1, df["position"])
        df["position"] = df["position"].ffill().fillna(0)
        
        self.data = df.copy()
        
    def execute_trades(self):
        if self.data["position"].iloc[-1] == 1:
            if self.position == 0:
                order = self.create_order(self.instrument, self.units, suppress=True, ret=True)
                self.report_trade(order, "GOING LONG")
            elif self.position == -1:
                order = self.create_order(self.instrument, self.units * 2, suppress=True, ret=True)
                self.report_trade(order, "GOING LONG")
            self.position = 1
        elif self.data["position"].iloc[-1] == -1:
            if self.position == 0:
                order = self.create_order(self.instrument, -self.units, suppress=True, ret=True)
                self.report_trade(order, "GOING SHORT")
            elif self.position == 1:
                order = self.create_order(self.instrument, -self.units * 2, suppress=True, ret=True)
                self.report_trade(order, "GOING SHORT")
            self.position = -1
        elif self.data["position"].iloc[-1] == 0:
            if self.position == -1:
                order = self.create_order(self.instrument, self.units, suppress=True, ret=True)
                self.report_trade(order, "GOING NEUTRAL")
            elif self.position == 1:
                order = self.create_order(self.instrument, -self.units, suppress=True, ret=True)
                self.report_trade(order, "GOING NEUTRAL")  
            self.position = 0
            
    def report_trade(self, order, going):
        time = order["time"]
        units = order["units"]
        price = order["price"]
        pl = float(order["pl"])
        self.profits.append(pl)
        cumpl = sum(self.profits)
        print("\n" + 100 * "-")
        print(f"{time} | {going}")
        print(f"{time} | units = {units} | price = {price} | P&L = {pl} | Cum P&L = {cumpl}")
        print(100 * "-" + "\n")

In [23]:
trader = DNNTrader("oanda.cfg", "EURUSD", bar_length = "20min",
                  window = 50, lags = 5, model = model, mu = mu, std = std, units = 50000)

In [24]:
trader.get_most_recent()
trader.stream_data(trader.instrument, stop = 100)
if trader.position != 0:
    close_order = trader.create_order(trader.instrument, units = -trader.position * trader.units,
                                     supress = True, ret = True)
    trader.report_trader(close_order, "GOING NEUTRAL")
    trader.position = 0

ResponseNoField: 401 response for GET https://api-fxpractice.oanda.com:443/v3/instruments/EURUSD/candles?price=M&granularity=M5&from=2025-03-10T23%3A07%3A43.000000000Z&to=2025-03-15T23%3A07%3A43.000000000Z does not have field 'candles' (contains 'errorMessage')

In [25]:
trader.data